In [146]:
import numpy as np
import plotly.graph_objects as go
theme = "plotly_dark"

In [147]:
def analytically_compute_probability_vec(transition_matrix):
    eigvals, eigvecs = np.linalg.eig(transition_matrix.T)
    mask = np.isclose(eigvals, 1)
    targ_eigvec = eigvecs[:, mask].ravel()
    probability_vec = targ_eigvec / sum(targ_eigvec)
    return probability_vec.real + 0

In [148]:
def numerically_compute_probability_vec(p, P, eps=0.0001, steps=1000, calculate_std=False):
    step = 0
    stds = []

    while not np.linalg.norm(p @ P - p) < eps and step <= steps:
        step += 1
        if calculate_std:
            stds.append(np.linalg.norm(p @ P - p))

        p = p @ P

    if calculate_std:
        return p / p.sum(), stds

    return p / p.sum()

In [151]:
P = np.array([
    [0.1, 0.2, 0.3, 0.4, 0, 0, 0, 0], 
    [0, 0, 0.25, 0, 0.25, 0, 0.5, 0], 
    [0, 0, 0, 0.5, 0, 0.1, 0, 0.4], 
    [0, 0, 0, 0.3, 0.4, 0, 0, 0.3], 
    [0, 0.1, 0, 0.2, 0, 0, 0.7, 0], 
    [0, 0.3, 0.4, 0, 0, 0, 0.3, 0], 
    [0, 0.1, 0, 0, 0, 0.8, 0.1, 0], 
    [0, 0, 0.4, 0, 0, 0.4, 0, 0.2],
])
P_old = np.array([
    [0.1, 0.3, 0, 0, 0.6, 0, 0, 0],
    [0.4, 0.1, 0, 0, 0.2, 0.1, 0.2, 0],
    [0, 0.1, 0.2, 0, 0, 0, 0.7, 0],
    [0, 0, 0, 0.1, 0, 0, 0.4, 0.5],
    [0.1, 0.2, 0, 0, 0.5, 0.2, 0, 0],
    [0, 0.1, 0, 0, 0.1, 0.2, 0.6, 0],
    [0, 0, 0.2, 0.2, 0, 0.3, 0.1, 0.2],
    [0, 0, 0, 0.2, 0, 0, 0.5, 0.3],
])
p1 = np.array([1., 0, 0, 0, 0, 0, 0, 0])
p2 = np.array([0, 0, 0, 0, 0, 0, 0, 1.])

num_prob1, stds1 = numerically_compute_probability_vec(p1, P, eps=0.00001, steps=1000, calculate_std=True)
num_prob2, stds2 = numerically_compute_probability_vec(p2, P, eps=0.00001, steps=1000, calculate_std=True)
an_prob = analytically_compute_probability_vec(P)

with np.printoptions(precision=4, suppress=True):
    print("Numerical 1:", num_prob1)
    print("Numerical 2:", num_prob2)
    print("Analytical:", an_prob)


Numerical 1: [0.     0.0908 0.1621 0.138  0.0779 0.2156 0.1829 0.1328]
Numerical 2: [0.     0.0908 0.1621 0.138  0.0779 0.2156 0.1829 0.1328]
Analytical: [0.     0.0908 0.1621 0.138  0.0779 0.2156 0.1829 0.1328]


In [152]:
fig = go.Figure(
    data=[
        go.Scatter(x=list(range(0, len(stds1))), y=stds1, name='Начало в 1'),
        go.Scatter(x=list(range(0, len(stds2))), y=stds2, name='Начало в 8')
    ],
    layout=go.Layout(
        template=theme,
        margin=dict(l=75, r=40, t=70, b=15),
        xaxis=dict(tickmode="linear", title="Шаг"),
        yaxis=dict(tickmode="linear", dtick=0.1, title="Отклонение"),
        legend=dict(x=0.8, y=0.95),
        width=750,
        height=550,
        title='Зависимость отклонения от шага'
    )
)
fig.show()